# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Nguyễn Quốc Huy
- MSSV: 20127188
- Lớp: 20CLC2

# Import

In [12]:
import pandas as pd
import numpy as np
from sklearn import model_selection

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [3]:
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)   

In [4]:
def mse(y, y_hat):
    return np.mean((y.ravel() - y_hat.ravel())**2)

# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [5]:
def task_a(X_train, y_train, X_test, y_test):
    weight = OLSLinearRegression().fit(X_train, y_train)
    y_predict = weight.predict(X_test)
    return np.sqrt(mse(y_test, y_predict)), np.array(weight.get_params())


In [8]:
RMSE, weight = task_a(X_train, y_train, X_test, y_test)
print('RMSE = ', RMSE)
print('Weight:', weight)

RMSE =  7.0640464305840505
Weight: [ 1.51013627e-02  9.02199807e-02  4.29218175e-02  1.39289117e-01
 -5.67332827e-01 -1.00765115e-04  7.40713438e-01  1.90935798e-01
  2.45059736e+01  2.39351661e+00]


<p style="text-align:center;"><b>Công thức hồi quy</p>

Life expectancy = 0.0151013627 * Adult Mortality + 0.0902199807 * BMI + 0.0429218175 * Polio + 0.139289117 * Diphtheria + (-0.567332827) * HIV/AIDS + (-0.000100765115) * GDP + 0.740713438 * Thinness age 10-19 + 0.190935798 * Thinness age 5-9 + 0.245059736 * Income composition of resources + 2.39351661 * Schooling

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [6]:
def task_b(X, y):
    arr_w = np.zeros(train.shape[1] - 1)
    arr_r = np.zeros(train.shape[1] - 1)

    for train_s, test_s in model_selection.KFold(n_splits = 5, shuffle = True).split(X):
        index = 0
        for i in range(0, 10):
            train_feature = X.iloc[train_s, i:i + 1]
            train_label = y.iloc[train_s]

            test_feature = X.iloc[test_s, i:i + 1]
            test_label = y.iloc[test_s]

            rmse, weight = task_a(train_feature, train_label, test_feature, test_label)

            arr_r[index] += rmse
            arr_w[index] += weight
            index += 1

    index_min = np.min(arr_r)
    for i in range(0, 10):
        if(arr_r[i] == index_min):
            position = i
    min = X_train.columns[position]
    return arr_r/5, arr_w/5, min

In [7]:
rmse, weight, min = task_b(X_train, y_train)
min_r = np.min(rmse)
RMSE = []
for i in range (0, 10):
    row=[]
    row.append(X_train.keys()[i])
    row.append(rmse[i])
    RMSE.append(row)
        
print("\nWeight: ", weight,"\nMin RMSE = ", min_r , "\nBest feature: ", min)
pd.DataFrame(RMSE,columns=['Đặc trưng','RMSE'])


Weight:  [2.47853709e-01 1.50572781e+00 7.85644849e-01 7.84454360e-01
 3.58571931e+00 3.03019663e-03 6.92955048e+00 6.88083517e+00
 1.04706939e+02 5.55740908e+00] 
Min RMSE =  11.761702307064917 
Best feature:  Schooling


,Đặc trưng,RMSE
0,Adult Mortality,46.259784
1,BMI,27.957454
2,Polio,17.938079
3,Diphtheria,16.052544
4,HIV/AIDS,67.061835
5,GDP,60.256001
6,Thinness age 10-19,51.789394
7,Thinness age 5-9,51.690806
8,Income composition of resources,13.314031
9,Schooling,11.761702


In [59]:
rmse_2, w = task_a(X_train.iloc[:,9:10], y_train, X_test.iloc[:,9:10], y_test)
print(rmse_2)
print(w)

10.260950391655376
[5.5573994]


Công thức hồi quy

$$\text{Life expectancy} = 5.5573994 * \text{Schooling} $$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

In [60]:
model = []
model_2 = []
index = []
for i in range(2,10):
    model.append(X_train.iloc[:,0:i])
    model_2.append(X_train.iloc[:,0:i]**3)
    index.append(i)

In [10]:
def task_c(X, y):
    arr_r = []

    for train_split, test_split in model_selection.KFold(n_splits = 5, shuffle = True).split(X):
        feature_train = X.iloc[train_split]
        label_train = y.iloc[train_split]

        feature_test = X.iloc[test_split]
        label_test = y.iloc[test_split]

        rmse, weight = task_a(feature_train, label_train, feature_test, label_test)

        arr_r.append(rmse)

    return np.mean(arr_r)

In [61]:
arr_m = []
arr_m2 = []
for i in model:
    RMSE = task_c(i, y_train)
    arr_m.append(RMSE)
for i in model_2:
    RMSE = task_c(i, y_train)
    arr_m.append(RMSE)
best_model = np.min(arr_m)
print("RMSE = ", best_model)


RMSE =  8.735078128055042


In [70]:
a=["2 đặc trưng(Adult Mortality,BMI)", "3 đặc trưng(Adult Mortality,BMI,Polio)", "4 đặc trưng(Adult Mortality,BMI,Polio,Diphtheria)","5 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS)", "6 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP)", "7 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19)","8 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19, Thinness age 5-9)","9 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19, Thinness age 5-9, Income composition of resources)","2 đặc trưng(Adult Mortality,BMI) mũ 3", "3 đặc trưng(Adult Mortality,BMI,Polio) mũ 3", "4 đặc trưng(Adult Mortality,BMI,Polio,Diphtheria) mũ 3","5 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS) mũ 3", "6 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP) mũ 3", "7 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19) mũ 3","8 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19, Thinness age 5-9) mũ 3","9 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19, Thinness age 5-9, Income composition of resources) mũ 3"]
# Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19, Thinness age 5-9, Income composition of resources, Schooling
data= list(zip([list for list in a], arr_m))
df = pd.DataFrame(w_data,columns=['Mô hình','RMSE'])
display_model = df.style.set_properties(**{'text-align': 'center'})
display_model = display_model.set_table_styles([dict(selector = 'th', props=[('text-align', 'center')])])
display(display_model)

,Mô hình,RMSE
0,"2 đặc trưng(Adult Mortality,BMI)",22.623468
1,"3 đặc trưng(Adult Mortality,BMI,Polio)",14.964050
2,"4 đặc trưng(Adult Mortality,BMI,Polio,Diphtheria)",13.147579
3,"5 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS)",12.990777
4,"6 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP)",12.891247
5,"7 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19)",11.675221
6,"8 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19, Thinness age 5-9)",11.539536
7,"9 đặc trưng(Adult Mortality, BMI, Polio, Diphtheria, HIV/AIDS, GDP, Thinness age 10-19, Thinness age 5-9, Income composition of resources)",8.735078
8,"2 đặc trưng(Adult Mortality,BMI) mũ 3",39.418300
9,"3 đặc trưng(Adult Mortality,BMI,Polio) mũ 3",23.246128


In [69]:
rmse_2, w = task_a(X_train.iloc[:,0:9], y_train, X_test.iloc[:,0:9], y_test)
print(rmse_2)
print(w)

7.3379852626993385
[ 1.73671084e-02  1.54066222e-01  7.80981602e-02  1.82294109e-01
 -4.32676946e-01 -5.40471095e-06  7.09662791e-01  2.60263436e-01
  5.36120768e+01]


<p style="text-align:center;"><b>Công thức hồi quy</p>

Life expectancy = 0.0173671084 * Adult Mortality + 0.0154066222 * BMI + 0.0780981602 * Polio + 0.182294109 * Diphtheria + (-0.432676946) * HIV/AIDS + (-0.00000540471095) * GDP + 0.709662791 * Thinness age 10-19 + 0. 260263436 * Thinness age 5-9 + 0.536120768 * Income composition of resources